## Explorations in how to improve performance

Calculating counts based on common spellings/pronunciations of similar names is very slow look at optimizing/parallelizing.

* see: https://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby
* potentially look at https://github.com/jmcarpenter2/swifter
* see realted blog post: https://medium.com/@jmcarpenter2/swiftapply-automatically-efficient-pandas-apply-operations-50e1058909f9



In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from timeit import default_timer as timer

In [2]:
%load_ext line_profiler

In [3]:
with open('babynames.pickle', 'rb') as f:
    orig_df = pickle.load(f)
orig_df.shape

(1924665, 5)

In [4]:
with open('names_beidermorse.pickle', 'rb') as f:
    names = pickle.load(f)
names.shape

(77092, 3)

In [5]:
# this takes about 35 seconds
start = timer()
def f(name, bmset):
    return pd.DataFrame(zip([name] * len(bmset), list(bmset)), columns=('name', 'beidermorse'))

kv_names = pd.concat([f(n,b) for n, b in zip(names['name'], names['bmset'])])
end = timer()
print(end - start, 'seconds')
print(kv_names.shape)

38.971319144000006 seconds
(1152268, 2)


In [6]:
# setup test dataset
df = orig_df[orig_df.year == 1990].sample(frac=.01, random_state=2213).copy()
#df = orig_df[orig_df.year == 1990].copy()
df.sort_values(['year', 'sex', 'name'], inplace=True)
df.shape

(247, 5)

```python
start = timer()
calc_sound_totals(alt)
end = timer()
print(end - start, 'seconds')
# right now it takes ~54 seconds to process 500 randomly selected records
# ~26 seconds to process 250 randomly selected records
# 95% or so of time is spend in the first block "out_n"
```

%lprun -f create_df_out_n calc_sound_totals(alt)

In [7]:
# this is very slow look at optimizing/parallelizing
# see: https://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby
# potentially look at https://github.com/jmcarpenter2/swifter
#   see realted blog post: https://medium.com/@jmcarpenter2/swiftapply-automatically-efficient-pandas-apply-operations-50e1058909f9

# process each row of dataframe
def create_df_out_n(row):
    # should do no further processing if this row has already been counted
    if (row['counted'] == True):
        return

    # find matching names
    #checklist = names[names.name == row['name']].beidermorse.values[0].split()
    #find = lambda i: any(x in i for x in checklist)
    #found = names[names.bmset.map(find)].name
    # this new method takes about half the time - 15 seconds for 250 rows
    #checklist = names[names.name == row['name']].bmset.values[0]
    #found = kv_names[kv_names.beidermorse.isin(checklist)]['name'].unique()
    
    checklist = filt_names[filt_names.name == row['name']].beidermorse.values
    found = filt_names[filt_names.beidermorse.isin(checklist)]['name'].unique()

    # aggregate count, excluding counted names, for all found names into df_out_name
    df_out.loc[(df_out.name == row['name']) &
            (df_out.year == row.year) &
            (df_out.sex == row.sex) ,
            'alt_n'] = df_out[(df_out.name.isin(found)) & 
                           (df_out.year == row.year) &
                           (df_out.sex == row.sex) &
                           (df_out.counted == False)]['n'].sum()

    # set counted flag for found names in group
    # ? how to update just group ?
    df_out.loc[(df_out.name.isin(found)) & (df_out.year == row.year) & (df_out.sex == row.sex), 'counted'] = True

# create df_out_prop
def create_df_out_prop(row, gsum):
    df_out.loc[(df_out.name == row['name']) &
            (df_out.year == row.year) &
            (df_out.sex == row.sex) ,
            'alt_prop'] = row['alt_n'] / gsum


    
def calc_sound_totals():
    gdf = df_out.groupby(['year', 'sex'])
    for name, group in gdf:
        print('processing name:', name)
        g = group.sort_values('n', ascending=False).copy()
        g.apply(create_df_out_n, axis=1)

    for name, group in gdf:
        gsum = group['alt_n'].sum()
        group.apply(create_df_out_prop, axis=1, args=(gsum,))

In [8]:
df_out = df.copy()
df_out['counted'] = False
df_out['alt_n'] = 0
df_out['alt_prop'] = 0.0

filt_names = kv_names.merge(df, on='name')[['name', 'beidermorse']]

#%lprun -f create_df_out_n calc_sound_totals()
start = timer()
out = calc_sound_totals()
end = timer()
print('took:', end - start, 'seconds')

processing name: (1990.0, 'F')
processing name: (1990.0, 'M')
took: 2.747541163000001 seconds


```python
# this is very slow look at optimizing/parallelizing
# see: https://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby
# potentially look at https://github.com/jmcarpenter2/swifter
#   see realted blog post: https://medium.com/@jmcarpenter2/swiftapply-automatically-efficient-pandas-apply-operations-50e1058909f9

def calc_sound_totals(df_in):
    df_out = df_in.copy()
    df_out['counted'] = False
    df_out['alt_n'] = 0
    df_out['alt_prop'] = 0.0

    # process each row of dataframe
    def create_df_out_n(row):
        # should do no further processing if this row has already been counted
        if (row['counted'] == True):
            return

        # find matching names
        checklist = names[names.name == row['name']].beidermorse.values[0].split()
        find = lambda i: any(x in i for x in checklist)
        found = names[names.bmset.map(find)].name

        # aggregate count, excluding counted names, for all found names into df_out_name
        df_out.loc[(df_out.name == row['name']) &
                (df_out.year == row.year) &
                (df_out.sex == row.sex) ,
                'alt_n'] = df_out[(df_out.name.isin(found)) & 
                               (df_out.year == row.year) &
                               (df_out.sex == row.sex) &
                               (df_out.counted == False)]['n'].sum()

        # set counted flag for found names in group
        # ? how to update just group ?
        df_out.loc[(df_out.name.isin(found)) & (df_out.year == row.year) & (df_out.sex == row.sex), 'counted'] = True

    start = timer()
    gdf = df_out.groupby(['year', 'sex'])
    for name, group in gdf:
        print('processing name:', name)
        g = group.sort_values('n', ascending=False).copy()
        g.apply(create_df_out_n, axis=1)

    end = timer()
    print('create out_n', end - start, 'seconds')
    
    # create df_out_prop
    def create_df_out_prop(row):
        df_out.loc[(df_out.name == row['name']) &
                (df_out.year == row.year) &
                (df_out.sex == row.sex) ,
                'alt_prop'] = row['alt_n'] / gsum

    start = timer()
    for name, group in gdf:
        gsum = group['alt_n'].sum()
        group.apply(create_df_out_prop, axis=1)
    end = timer()
    print('create out_prop', end - start, 'seconds')

    return df_out
```


In [9]:
%%timeit
rn = 'Michael'
checklist = names[names.name == rn].bmset.values[0]
found = kv_names[kv_names.beidermorse.isin(checklist)]['name'].unique()

42.2 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
rn = 'Michael'
checklist = names[names.name == rn].beidermorse.values[0].split()
find = lambda i: any(x in i for x in checklist)
found = names[names.bmset.map(find)].name

105 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
rn = 'Michael'
checklist = kv_names[kv_names.name == rn].beidermorse
found = kv_names[kv_names.beidermorse.isin(checklist)]['name'].unique()

86.6 ms ± 746 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
kn = kv_names.values

In [13]:
%%timeit
rn = 'Michael'
bm = kn[np.where(kn == rn)[0]][:,1]
out = [kn[np.where(kn == x)[0]][:,0] for x in bm]
found = np.unique(np.concatenate( out, axis=0 ))

562 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
ni = kv_names.set_index('name')
bi = kv_names.set_index('beidermorse')

In [15]:
%%timeit
rn = 'Michael'
found = bi.loc[ni.loc[rn].beidermorse].name.unique()

46 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
rn = 'Michael'
checklist = kv_names[kv_names.name == rn].beidermorse
found = kv_names.merge(checklist, on='beidermorse')['name'].unique()

338 ms ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# if I filter the list to just those in this year/group, performance is much improved.
sub = kv_names.merge(df, on='name')[['name', 'beidermorse']]

In [18]:
%%timeit
rn = 'Michael'
checklist = sub[sub.name == rn].beidermorse.values
found = sub[sub.beidermorse.isin(checklist)]['name'].unique()

1.23 ms ± 42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%prun -l 10 kv_names[kv_names.beidermorse.isin(['zYsDki'])]['name'].unique()

         1316 function calls (1296 primitive calls) in 0.029 seconds

   Ordered by: internal time
   List reduced from 229 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.026    0.026    0.026    0.026 {pandas._libs.hashtable.ismember_object}
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.026    0.026 algorithms.py:407(<lambda>)
      239    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       16    0.000    0.000    0.000    0.000 dtypes.py:68(find)
       47    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
       26    0.000    0.000    0.000    0.000 common.py:1845(_is_dtype_type)
        1    0.000    0.000    0.029    0.029 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'unique' of 'pandas._libs.hashtable.StringHashTable' objects}
        1    0.000 

In [20]:
checklist = kv_names[kv_names.name == 'Michael'].beidermorse
%prun -l 10 kv_names[kv_names.beidermorse.isin(checklist)]['name'].unique()

         1392 function calls (1368 primitive calls) in 0.042 seconds

   Ordered by: internal time
   List reduced from 228 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.040    0.040    0.040    0.040 {pandas._libs.hashtable.ismember_object}
        1    0.000    0.000    0.040    0.040 algorithms.py:407(<lambda>)
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
      254    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       18    0.000    0.000    0.000    0.000 dtypes.py:68(find)
       47    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
        1    0.000    0.000    0.042    0.042 {built-in method builtins.exec}
     13/9    0.000    0.000    0.000    0.000 {built-in method numpy.array}
       26    0.000    0.000    0.000    0.000 common.py:1845(_is_dtype_type)
        1    0.000    0.000    0.000    0.000 {pandas._libs

In [21]:
%%timeit
sub = kv_names.merge(df, on='name')[['name', 'beidermorse']]

67.3 ms ± 678 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
